In [1]:
import numpy as np
import numpy.typing as npt
from numba import jit, njit, prange
import pickle
from dataclasses import dataclass, asdict, field
import argparse
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd
from pathlib import Path
from typing import Any
import time
import sys
import copy
import concurrent.futures as cf

from src.dataclass import (
    Input, Lattice, Parameter, Train, Save,
    Processed_Input, Topology, Conjugate, Result
)
from src.manage_data import save_result, save_log, load_result
from src.initial_state import get_initial_state
from src.process_input import (get_processed_input, get_T_and_H, get_J)
from src.metropolis import execute_metropolis_update
from src.function import (time_correlation, column_average_2d, get_spin_glass)
from src.process_output import get_result

In [6]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

"""
Memory
Max(measurement * size**dim, size**(2*dim)) * max_workers

Performance
measurement * size**dim * ensembles / max_workers
"""

"""
Lattice Condition
"""
args.state = 3
args.size = 2**6
args.dimension = 2
args.ghost = 0
args.initial = "uniform" # "uniform", "random"

"""
Parameter Condition
"""
args.T, args.H = 1.0, 0.0
# q=2: 2.2692, q=3: 1.4925
# args.Tc = 1/((1-1/args.state)*np.log(1+np.sqrt(args.state)))
args.Tc = 1.5
args.Hc = 0.0
args.Jm = 1.0
args.Jv = 1.0

args.mode = "normal"  # 'normal', 'critical' and 'manual'
args.variable = "T"  # 'T', 'H'
args.multiply = 0.1**4
args.base = 2.0
args.exponent = 13

"""
Train Condition
"""
args.iteration = 2**10
args.measurement = 2**14
args.interval = 2**3
args.sweep = args.measurement * args.interval
args.ensemble = 2**3
args.max_workers = 2**3
args.threshold = 5 * 0.1**2
args.recent = 10**2

"""
Save Condition
"""
args.environment = "server" # "server" or "local"
args.save = True  # True or False

lattice = Lattice(
args.state, args.size, args.dimension, 
args.ghost, args.initial)
parameter = Parameter(
args.Tc, args.Hc, args.Jm, args.Jv, args.mode, 
args.variable, args.multiply, args.base, args.exponent)
train = Train(
args.iteration, args.sweep, args.measurement, args.interval,
args.ensemble, args.max_workers, args.threshold, args.recent)
save = Save(args.environment, args.save)

input = Input(lattice, parameter, train, save)
input.parameter.T, input.parameter.H = args.T, args.H

processed_input = get_processed_input(input)
input.parameter.T, input.parameter.H = get_T_and_H(input)
J = get_J(input, processed_input)
print(input)

Input(lattice=Lattice(state=3, size=64, dimension=2, ghost=0, initial='uniform'), parameter=Parameter(T=2.3192000000000004, H=0.0, Tc=1.5, Hc=0.0, Jm=1.0, Jv=1.0, mode='normal', variable='T', multiply=0.00010000000000000002, base=2.0, exponent=13), train=Train(iteration=1024, sweep=131072, measurement=16384, interval=8, ensemble=8, max_workers=8, threshold=0.05000000000000001, recent=100), save=Save(environment='server', save=True))
